<a href="https://colab.research.google.com/github/kmorawala/Alexa-Interview-Me/blob/master/data_cleaning/Data_Cleaning_and_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Items to discuss:
Per BCNF, I think we may have to further breakdown the tables:
* I suggest  removing the park ID in the parks data


## Setup / Importing necessary libraries


In [ ]:
import requests
import json
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import re

## Mounting Google Drive

In [ ]:
prefix = '/content/drive'
from google.colab import drive
drive.mount(prefix, force_remount=True)

Mounted at /content/drive


## Set up file paths

In [ ]:
parks_path = '/content/drive/Shareddrives/CIS 550 Project/parks.csv' # Path
species_path = '/content/drive/Shareddrives/CIS 550 Project/species.csv' # Path
visitation_data_path = '/content/drive/Shareddrives/CIS 550 Project/visitation_data.csv' # Path
visitation_data = pd.read_csv(visitation_data_path,skipinitialspace=True)
species_data = pd.read_csv(species_path,skipinitialspace=True)
park_data = pd.read_csv(parks_path,skipinitialspace=True)
visitation_data = visitation_data.replace(r'^\s*$', np.NaN, inplace=False, regex=True)
species_data = species_data.replace(r'^\s*$', np.NaN, inplace=False, regex=True)
park_data = park_data.replace(r'^\s*$', np.NaN, inplace=False, regex=True)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# View any rows with NaN values
visitation_data.loc[:, visitation_data.isna().any()]

,Metadata,Parkname,Visitors
0,NaN,Crater Lake,1500.0
1,NaN,Lake Roosevelt,0.0
2,NaN,Lewis and Clark,69000.0
3,NaN,Olympic,2200.0
4,NaN,Santa Monica Mountains,468144.0
...,...,...,...
21555,NaN,Crater Lake,1400.0
21556,http://nrdata.nps.gov/programs/Lands/YELL_meta...,Yellowstone,26188.0
21557,http://nrdata.nps.gov/programs/Lands/KICA_meta...,Kings Canyon,1000.0
21558,http://nrdata.nps.gov/programs/Lands/MORA_meta...,Mount Rainier,928.0


In [ ]:
#view items with NA
park_data.loc[:, park_data.isna().any()]

""
0
1
2
3
4
5
6
7
8
9


In [ ]:
#view items with NA
species_data.loc[:, species_data.isna().any()]

,Order,Family,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status,Unnamed: 14
0,Artiodactyla,Cervidae,Present,Native,Rare,Resident,NaN,NaN
1,Artiodactyla,Cervidae,Present,Native,Abundant,NaN,NaN,NaN
2,Carnivora,Canidae,Present,Not Native,Common,NaN,Species of Concern,NaN
3,Carnivora,Canidae,Not Confirmed,Native,NaN,NaN,Endangered,NaN
4,Carnivora,Canidae,Present,Unknown,Common,Breeder,NaN,NaN
...,...,...,...,...,...,...,...,...
119243,Solanales,Solanaceae,Present,Native,Uncommon,NaN,NaN,NaN
119244,Vitales,Vitaceae,Present,Native,Uncommon,NaN,NaN,NaN
119245,Vitales,Vitaceae,Present,Not Native,Uncommon,NaN,NaN,NaN
119246,Zygophyllales,Zygophyllaceae,Present,Native,Rare,NaN,NaN,NaN


# Data Cleaning and Extractions

## View CSV files

In [ ]:
visitation_data.head()

,Created By,Measure Selector,Year,Date Edit,ScrapeURL,GIS Notes,Gnis Id,Geometry,Metadata,Number of Records,Parkname,Region,State,Unit Code,Unit Name,Unit Type,Visitors,YearRaw
0,Legacy,1,1/1/1904 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,LEGACY: contact allen_mccoy@nps.gov for additi...,1163670,POLYGON,NaN,1,Crater Lake,PW,OR,CRLA,Crater Lake National Park,National Park,1500.0,1904
1,Legacy,1,1/1/1941 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,LEGACY: contact allen_mccoy@nps.gov for additi...,1531834,MULTIPOLYGON,NaN,1,Lake Roosevelt,PW,WA,LARO,Lake Roosevelt National Recreation Area,National Recreation Area,0.0,1941
2,Legacy,1,1/1/1961 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,LEGACY: contact allen_mccoy@nps.gov for additi...,2055170,MULTIPOLYGON,NaN,1,Lewis and Clark,PW,WA,LEWI,Lewis and Clark National Historical Park,National Historical Park,69000.0,1961
3,Legacy,1,1/1/1935 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,LEGACY: contact allen_mccoy@nps.gov for additi...,1530459,MULTIPOLYGON,NaN,1,Olympic,PW,WA,OLYM,Olympic National Park,National Park,2200.0,1935
4,Legacy,1,1/1/1982 12:00:00 AM,4/22/2015,https://irma.nps.gov/Stats/SSRSReports/Park%20...,LEGACY: contact denise_kamradt@nps.gov for add...,277263,POLYGON,NaN,1,Santa Monica Mountains,PW,CA,SAMO,Santa Monica Mountains National Recreation Area,National Recreation Area,468144.0,1982


In [ ]:
park_data.head()

,Park Code,Park Name,State,Acres,Latitude,Longitude
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


In [ ]:
species_data.head()

,Species ID,Park Name,Park_code,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status,Unnamed: 14
0,ACAD-1000,Acadia National Park,ACAD,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,NaN,NaN
1,ACAD-1001,Acadia National Park,ACAD,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,NaN,NaN,NaN
2,ACAD-1002,Acadia National Park,ACAD,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,NaN,Species of Concern,NaN
3,ACAD-1003,Acadia National Park,ACAD,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,NaN,NaN,Endangered,NaN
4,ACAD-1004,Acadia National Park,ACAD,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,NaN,NaN


## Clean species data and make it import ready

In [ ]:
# Remove unnecessary columns
cols_to_drop = ['Order', 'Park Name','Family','Record Status', 'Abundance', 'Seasonality', 'Conservation Status','Unnamed: 14']
species_data.drop(columns=cols_to_drop,inplace=True)
# species_data.head()

# remove all rows with missing values in all columns
cols = ["Species ID","Park_code","Category","Scientific Name","Common Names","Occurrence","Nativeness"]
# print(species_data.columns)
# Only keep the data related to mammals and birds
species = species_data.query('Category == "Mammal" or Category == "Bird"')
# species = species.concat([pd.Series(row['Species ID'], row['Common Names'].split(',')) for _, row in species.iterrows()]).reset_index()


# split the column "common names" by a comma and add each item as a row
species = species.set_index(['Species ID',"Park_code","Category","Scientific Name","Occurrence","Nativeness"]).apply(lambda x: x.str.split(',').explode()).reset_index()

# Empty items in Occurrence and Nativeness should be changed to 'Not Confirmed' and 'Unknown'
species[['Occurrence']].fillna(value='Not Confirmed', inplace=True)
species[['Nativeness']].fillna(value='Unknown', inplace=True)
species['Common Names'] = species['Common Names'].str.strip()
species['Common Names'] = species['Common Names'].map(lambda x: re.sub(r'[^\w\s]', '', x))

missing = species_data[cols].copy()
for col in cols:
  missing[col] = missing[col].str.contains('?', regex=False)
missing = missing.any(axis=1)
missing = missing.index[missing]
species_data = species_data.drop(labels=missing)

species.sort_values(by=['Species ID', 'Category'], inplace=True)
species.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,Species ID,Park_code,Category,Scientific Name,Occurrence,Nativeness,Common Names
0,ACAD-1000,ACAD,Mammal,Alces alces,Present,Native,Moose
1,ACAD-1001,ACAD,Mammal,Odocoileus virginianus,Present,Native,Northern WhiteTailed Deer
2,ACAD-1001,ACAD,Mammal,Odocoileus virginianus,Present,Native,Virginia Deer
3,ACAD-1001,ACAD,Mammal,Odocoileus virginianus,Present,Native,WhiteTailed Deer
4,ACAD-1002,ACAD,Mammal,Canis latrans,Present,Not Native,Coyote


### Create import template

In [ ]:
#view items with NA
species_data.loc[:, species_data.isna().any()]

,Occurrence,Nativeness
0,Present,Native
1,Present,Native
2,Present,Not Native
3,Not Confirmed,Native
4,Present,Unknown
...,...,...
119243,Present,Native
119244,Present,Native
119245,Present,Not Native
119246,Present,Native


In [ ]:
# Write to the CSV import file
species.to_csv(r'/content/drive/Shareddrives/CIS 550 Project/Import Files/species_import.csv', index=False)

## Clean visitation data CSV table and make it import ready

In [ ]:
# keep only necessary columns
cols_to_keep = ['Unit Code', 'Unit Type', 'Visitors','YearRaw']
visitation_data = visitation_data[cols_to_keep]
# visitation_data.head()

# remove all rows with missing values in all columns
cols = ['Unit Code']
missing = visitation_data[cols].copy()
for col in cols:
  missing[col] = missing[col].str.contains('?', regex=False)
missing = missing.any(axis=1)
missing = missing.index[missing]
visitation_data = visitation_data.drop(labels=missing)

# remove all rows with missing values in all columns
cols = ['Visitors']
for col in cols:
  missing = visitation_data.index[visitation_data[col].isna()]
  visitation_data.loc[missing, col] = 0

# Remove the rows that has totals
visitation_data = visitation_data[visitation_data.YearRaw != 'Total']

visitation_data.rename(columns={"Unit Code": "parkcode", 
                                "Unit Type":"unit_type",
                            "Visitors": "total_visitors", 
                            "YearRaw": "year"}, inplace=True)

visitation_data.sort_values(by=['parkcode'], inplace=True)

# Only keep the data related to mammals and birds
visitation_data.head()

,parkcode,unit_type,total_visitors,year
11015,ABLI,National Historical Park,292825.0,1987
17408,ABLI,National Historical Park,331300.0,1960
13614,ABLI,National Historical Park,355687.0,1978
1621,ABLI,National Historical Park,241264.0,2014
99,ABLI,National Historical Park,175000.0,1934


### Create import template

In [ ]:
# Write to the CSV import file
visitation_data.to_csv(r'/content/drive/Shareddrives/CIS 550 Project/Import Files/visitation_data_import.csv', index=False)

## Clean park info data CSV table and make it import ready

### Get the list of park codes for API call data extraction

In [ ]:
list_of_parkcodes = []
try:
    file = open(parks_path, 'r')
    lines = file.readlines()[1:]
    for line in lines:
        # print(line)
        words = line.split(",")
        park_code = words[0]
        # print(park_code)
        list_of_parkcodes.append(park_code)
    file.close()
except:
    print("Error! File cannot be opened!")
    exit()

In [ ]:
print(list_of_parkcodes)

['ACAD', 'ARCH', 'BADL', 'BIBE', 'BISC', 'BLCA', 'BRCA', 'CANY', 'CARE', 'CAVE', 'CHIS', 'CONG', 'CRLA', 'CUVA', 'DENA', 'DEVA', 'DRTO', 'EVER', 'GAAR', 'GLAC', 'GLBA', 'GRBA', 'GRCA', 'GRSA', 'GRSM', 'GRTE', 'GUMO', 'HALE', 'HAVO', 'HOSP', 'ISRO', 'JOTR', 'KATM', 'KEFJ', 'KOVA', 'LACL', 'LAVO', 'MACA', 'MEVE', 'MORA', 'NOCA', 'OLYM', 'PEFO', 'PINN', 'REDW', 'ROMO', 'SAGU', 'SEKI', 'SHEN', 'THRO', 'VOYA', 'WICA', 'WRST', 'YELL', 'YOSE', 'ZION']


### Get the API response set-up


In [ ]:
api_url = "https://developer.nps.gov/api/v1/parks?parkCode=" #general park info
# api_url = "https://developer.nps.gov/api/v1/thingstodo?parkCode=" #specific activities, including trails
param = "&api_key="

#### Function to flatten nested JSON objects



In [ ]:
# Source: https://stackoverflow.com/questions/52795561/flattening-nested-json-in-pandas-data-frame
def flatten_json(nested_json, exclude=['']):
  """Flatten json object with nested keys into a single level.
        Args:
            nested_json: A nested json object.
            exclude: Keys to exclude from output.
        Returns:
            The flattened json object if successful, None otherwise."""
  out = {}
  # print("in flatten_json")

  def flatten(obj, name='', exclude=exclude):
      if type(obj) is dict:
        for key in obj:
          if key not in exclude:
            flatten(obj[key], name + key + '_')
      elif type(obj) is list:
          i = 0
          for item in obj:
            flatten(item, name + str(i) + '_')
            i += 1
      else:
        out[name[:-1]] = obj

  flatten(nested_json)
  return out


#### Flattened JSON objects and create dataframe

In [ ]:
parks_info = pd.DataFrame()

for park_code in list_of_parkcodes:
  # key = input("Enter key:")
  key = "rpm9HIDzfJhHfpIFnDpDCzSWRhFl1zCLw37RN1O7"
  url_text = f'{api_url}{park_code}{param}{key}'
  res = requests.get(url=url_text)
  data = res.json()
  flattened = flatten_json(data)
  park_info = pd.json_normalize(flattened)
  parks_info = parks_info.append(park_info)
  parks_info.head()

In [ ]:
parks_info.head()

,total,limit,start,data_0_id,data_0_url,data_0_fullName,data_0_parkCode,data_0_description,data_0_latitude,data_0_longitude,data_0_latLong,data_0_activities_0_id,data_0_activities_0_name,data_0_activities_1_id,data_0_activities_1_name,data_0_activities_2_id,data_0_activities_2_name,data_0_activities_3_id,data_0_activities_3_name,data_0_activities_4_id,data_0_activities_4_name,data_0_activities_5_id,data_0_activities_5_name,data_0_activities_6_id,data_0_activities_6_name,data_0_activities_7_id,data_0_activities_7_name,data_0_activities_8_id,data_0_activities_8_name,data_0_activities_9_id,data_0_activities_9_name,data_0_activities_10_id,data_0_activities_10_name,data_0_activities_11_id,data_0_activities_11_name,data_0_activities_12_id,data_0_activities_12_name,data_0_activities_13_id,data_0_activities_13_name,data_0_activities_14_id,...,data_0_images_28_credit,data_0_images_28_title,data_0_images_28_altText,data_0_images_28_caption,data_0_images_28_url,data_0_images_29_credit,data_0_images_29_title,data_0_images_29_altText,data_0_images_29_caption,data_0_images_29_url,data_0_operatingHours_1_exceptions_1_exceptionHours_wednesday,data_0_operatingHours_1_exceptions_1_exceptionHours_monday,data_0_operatingHours_1_exceptions_1_exceptionHours_thursday,data_0_operatingHours_1_exceptions_1_exceptionHours_sunday,data_0_operatingHours_1_exceptions_1_exceptionHours_tuesday,data_0_operatingHours_1_exceptions_1_exceptionHours_friday,data_0_operatingHours_1_exceptions_1_exceptionHours_saturday,data_0_operatingHours_1_exceptions_1_startDate,data_0_operatingHours_1_exceptions_1_name,data_0_operatingHours_1_exceptions_1_endDate,data_0_operatingHours_3_exceptions_1_exceptionHours_wednesday,data_0_operatingHours_3_exceptions_1_exceptionHours_monday,data_0_operatingHours_3_exceptions_1_exceptionHours_thursday,data_0_operatingHours_3_exceptions_1_exceptionHours_sunday,data_0_operatingHours_3_exceptions_1_exceptionHours_tuesday,data_0_operatingHours_3_exceptions_1_exceptionHours_friday,data_0_operatingHours_3_exceptions_1_exceptionHours_saturday,data_0_operatingHours_3_exceptions_1_startDate,data_0_operatingHours_3_exceptions_1_name,data_0_operatingHours_3_exceptions_1_endDate,data_0_operatingHours_4_exceptions_1_exceptionHours_wednesday,data_0_operatingHours_4_exceptions_1_exceptionHours_monday,data_0_operatingHours_4_exceptions_1_exceptionHours_thursday,data_0_operatingHours_4_exceptions_1_exceptionHours_sunday,data_0_operatingHours_4_exceptions_1_exceptionHours_tuesday,data_0_operatingHours_4_exceptions_1_exceptionHours_friday,data_0_operatingHours_4_exceptions_1_exceptionHours_saturday,data_0_operatingHours_4_exceptions_1_startDate,data_0_operatingHours_4_exceptions_1_name,data_0_operatingHours_4_exceptions_1_endDate
0,1,50,0,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,https://www.nps.gov/acad/index.htm,Acadia National Park,acad,Acadia National Park protects the natural beau...,44.409286,-68.247501,"lat:44.409286, long:-68.247501",09DF0950-D319-4557-A57E-04CD2F63FF42,Arts and Culture,FAED7F97-3474-4C21-AB42-FB0768A2F826,Cultural Demonstrations,13A57703-BB1A-41A2-94B8-53B692EB7238,Astronomy,D37A0003-8317-4F04-8FB0-4CF0A272E195,Stargazing,7CE6E935-F839-4FEC-A63E-052B1DEF39D2,Biking,071BA73C-1D3C-46D4-A53C-00D5602F7F0E,Boating,A59947B7-3376-49B4-AD02-C0423E08C5F7,Camping,7CFF5F03-5ECC-4F5A-8572-75D1F0976C0C,Group Camping,B12FAAB9-713F-4B38-83E4-A273F5A43C77,Climbing,87D3B1CD-3903-4561-ABB1-2DD870C43F2F,Rock Climbing,C11D3746-5063-4BD0-B245-7178D1AD866C,Compass and GPS,CA3641A0-FADC-497F-B036-3FE426D0DDCC,Geocaching,AE42B46C-E4B7-4889-A122-08FE180371AE,Fishing,25FB188F-5AAD-459A-9092-28A9801709FF,Freshwater Fishing,9BC03FAF-28F1-4609-BF6F-643F58071AED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1,50,0,36240051-018E-4915-B6EA-3F1A7F24FBE4,https://www.nps.gov/arch/index.htm,Arches National Park,arch,Visit Arches to discover a landscape of

### Initial data clean-up on park_info (Converting from spreadsheet style data)

In [ ]:
# Dropping unnecessary columns
parks_info.drop(columns=["total","limit", "start","data_0_latLong"],inplace=True)

# Removing "data_0_" prefix from all column names
parks_info.columns = parks_info.columns.str[7:]

# Removing all columns containing words "topics" and "operatingHours"
cols_to_remove = ['topics', 'operatingHours']
for word in cols_to_remove:
  parks_info = parks_info[parks_info.columns.drop(list(parks_info.filter(regex=word)))]
# parks_info.head()

# Rename columns
parks_info.rename(columns={"url": "website", 
                            "fullName": "parkname", 
                            "parkCode": "parkcode", 
                            "contacts_phoneNumbers_0_phoneNumber": "phone_number", 
                            "contacts_emailAddresses_0_emailAddress": "email", 
                            "weatherInfo": "weather", 
                            "entranceFees_0_cost": "entrance_fees_vehicle", 
                            "entranceFees_1_cost": "entrance_fees_motorcycle", 
                            "entranceFees_2_cost": "entrance_fees_individuals", 
                            "entrancePasses_0_cost":"entrance_pass_fees", 
                            "directionsInfo":"direction",
                            "addresses_0_line1":"address_line_1",
                            "addresses_0_line2":"address_line_2",
                            "addresses_0_line3":"address_line_3",
                            "addresses_0_stateCode":"state",
                            "addresses_0_city":"city",
                            "addresses_0_postalCode":"zipcode"}, inplace=True)

cols_to_remove = ['contacts', 'entranceFees', 'entrancePasses', 'addresses']
parks_info = parks_info.replace(r'^\s*$', np.NaN, inplace=False, regex=True)

# Replace the empty cells with 0  for  entrance_fees_motorcycle, entrance_fees_individuals, entrance_pass_fees
# because the entry is free per the visual inspection
parks_info[['entrance_fees_motorcycle','entrance_fees_individuals', 'entrance_pass_fees']].fillna(value=0, inplace=True)

# TODO: address_line_2 and address_line_3: empty cells should be replaced by "N/A" since some parks will not address line 2 and 3
parks_info[['address_line_2', 'address_line_3']].fillna(value='N/A', inplace=True)

# Removing all columns containing words
for word in cols_to_remove:
  parks_info = parks_info[parks_info.columns.drop(list(parks_info.filter(regex=word)))]
parks_info.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,id,website,parkname,parkcode,description,latitude,longitude,activities_0_id,activities_0_name,activities_1_id,activities_1_name,activities_2_id,activities_2_name,activities_3_id,activities_3_name,activities_4_id,activities_4_name,activities_5_id,activities_5_name,activities_6_id,activities_6_name,activities_7_id,activities_7_name,activities_8_id,activities_8_name,activities_9_id,activities_9_name,activities_10_id,activities_10_name,activities_11_id,activities_11_name,activities_12_id,activities_12_name,activities_13_id,activities_13_name,activities_14_id,activities_14_name,activities_15_id,activities_15_name,activities_16_id,...,images_22_credit,images_22_title,images_22_altText,images_22_caption,images_22_url,images_23_credit,images_23_title,images_23_altText,images_23_caption,images_23_url,images_24_credit,images_24_title,images_24_altText,images_24_caption,images_24_url,images_25_credit,images_25_title,images_25_altText,images_25_caption,images_25_url,images_26_credit,images_26_title,images_26_altText,images_26_caption,images_26_url,images_27_credit,images_27_title,images_27_altText,images_27_caption,images_27_url,images_28_credit,images_28_title,images_28_altText,images_28_caption,images_28_url,images_29_credit,images_29_title,images_29_altText,images_29_caption,images_29_url
0,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,https://www.nps.gov/acad/index.htm,Acadia National Park,acad,Acadia National Park protects the natural beau...,44.409286,-68.247501,09DF0950-D319-4557-A57E-04CD2F63FF42,Arts and Culture,FAED7F97-3474-4C21-AB42-FB0768A2F826,Cultural Demonstrations,13A57703-BB1A-41A2-94B8-53B692EB7238,Astronomy,D37A0003-8317-4F04-8FB0-4CF0A272E195,Stargazing,7CE6E935-F839-4FEC-A63E-052B1DEF39D2,Biking,071BA73C-1D3C-46D4-A53C-00D5602F7F0E,Boating,A59947B7-3376-49B4-AD02-C0423E08C5F7,Camping,7CFF5F03-5ECC-4F5A-8572-75D1F0976C0C,Group Camping,B12FAAB9-713F-4B38-83E4-A273F5A43C77,Climbing,87D3B1CD-3903-4561-ABB1-2DD870C43F2F,Rock Climbing,C11D3746-5063-4BD0-B245-7178D1AD866C,Compass and GPS,CA3641A0-FADC-497F-B036-3FE426D0DDCC,Geocaching,AE42B46C-E4B7-4889-A122-08FE180371AE,Fishing,25FB188F-5AAD-459A-9092-28A9801709FF,Freshwater Fishing,9BC03FAF-28F1-4609-BF6F-643F58071AED,Fly Fishing,17411C8D-5769-4D0F-ABD1-52ED03840C18,Saltwater Fishing,1DFACD97-1B9C-4F5A-80F2-05593604799E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,36240051-018E-4915-B6EA-3F1A7F24FBE4,https://www.nps.gov/arch/index.htm,Arches National Park,arch,Visit Arches to discover a landscape of contra...,38.72261844,-109.5863666,09DF0950-D319-4557-A57E-04CD2F63FF42,Arts and Culture,5F723BAD-7359-48FC-98FA-631592256E35,Auto and ATV,0B4A5320-216D-451A-9990-626E1D5ACE28,Scenic Driving,13A57703-BB1A-41A2-94B8-53B692EB7238,Astronomy,D37A0003-8317-4F04-8FB0-4CF0A272E195,Stargazing,7CE6E935-F839-4FEC-A63E-052B1DEF39D2,Biking,A59947B7-3376-49B4-AD02-C0423E08C5F7,Camping,4A58AF13-E8FB-4530-B41A-97DF0B0C77B7,Backcountry Camping,9159DF0F-951D-4AAE-9987-CEB3CE2A9ADA,Car or Front Country Camping,7CFF5F03-5ECC-4F5A-8572-75D1F0976C0C,Group Camping,07CBCA6A-46B8-413F-8B6C-ABEDEBF9853E,Canyoneering,B12FAAB9-713F-4B38-83E4-A273F5A43C77,Climbing,87D3B1CD-3903-4561-ABB1-2DD870C43F2F,Rock Climbing,B33DC9B6-0B7D-4322-BAD7-A13A34C584A3,Guided Tours,BFF8C027-7C8F-480B-A5F8-CD8CE490BFBA,Hiking,0307955A-B65C-4CE4-A780-EB36BAAADF0B,Horse Trekking,1886DA47-0AEC-4568-B9C2-8E9BC316AAAC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,B170CCF7-7AB9-48FF-950E-31815FD4DBB2,https://www.nps.gov/badl/index.htm,Badlands National Park,badl,The rugged beauty of the Badlands draws visito...,43.68584846,-102.482942,5F723BAD-7359-48FC-98FA-631592256E35,Auto and ATV,0B4A5320-216D-451A-9990-626E1D5ACE28,Scenic Driving,13A57703-BB1A-41A2-94B8-53B692EB7238,Astronomy,7CE6E935-F839-4FEC-A

## Extracting activities data out of the spreadsheet style dataframe into activities dataframe

In [ ]:
activities_cols = ["parkcode"]
activities_cols.extend([col for col in parks_info if 'activities' in col]) 
activities = parks_info[activities_cols]
activities.head()

activities_df = pd.DataFrame()
for i in range(56):
  activity_name = "activities_" + str(i) + "_name"
  temp = activities[['parkcode', activity_name]]
  temp = temp.rename(columns={activity_name: "activity"})
  # print(temp)
  activities_df = activities_df.append(temp)
# activities_df.head(25)
activities_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
activities_df.sort_values(by=['parkcode', 'activity'], inplace=True)
activities_df.head(100)

,parkcode,activity
0,acad,Arts and Culture
0,acad,Astronomy
0,acad,Biking
0,acad,Birdwatching
0,acad,Boat Tour
...,...,...
0,bibe,Canoeing
0,bibe,Fishing
0,bibe,Food
0,bibe,Guided Tours


In [ ]:
parks_info = parks_info[parks_info.columns.drop(list(parks_info.filter(regex='activities')))]
parks_info.head()

,id,website,parkname,parkcode,description,latitude,longitude,states,phone_number,email,entrance_fees_vehicle,entrance_fees_motorcycle,entrance_fees_individuals,entrance_pass_fees,direction,directionsUrl,zipcode,city,state,address_line_1,address_line_3,address_line_2,images_0_credit,images_0_title,images_0_altText,images_0_caption,images_0_url,images_1_credit,images_1_title,images_1_altText,images_1_caption,images_1_url,images_2_credit,images_2_title,images_2_altText,images_2_caption,images_2_url,images_3_credit,images_3_title,images_3_altText,...,images_22_credit,images_22_title,images_22_altText,images_22_caption,images_22_url,images_23_credit,images_23_title,images_23_altText,images_23_caption,images_23_url,images_24_credit,images_24_title,images_24_altText,images_24_caption,images_24_url,images_25_credit,images_25_title,images_25_altText,images_25_caption,images_25_url,images_26_credit,images_26_title,images_26_altText,images_26_caption,images_26_url,images_27_credit,images_27_title,images_27_altText,images_27_caption,images_27_url,images_28_credit,images_28_title,images_28_altText,images_28_caption,images_28_url,images_29_credit,images_29_title,images_29_altText,images_29_caption,images_29_url
0,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,https://www.nps.gov/acad/index.htm,Acadia National Park,acad,Acadia National Park protects the natural beau...,44.409286,-68.247501,ME,2072883338,acadia_information@nps.gov,30.00,25.00,15.00,55.00,"From Boston take I-95 north to Augusta, Maine,...",http://www.nps.gov/acad/planyourvisit/directio...,04609,Bar Harbor,ME,25 Visitor Center Road,NaN,Hulls Cove Visitor Center,NPS / Kristi Rugg,Acadia's rocky coastline,Large puffy clouds dot a brilliant blue sky as...,Millions of people come to Acadia for our dist...,https://www.nps.gov/common/uploads/structured_...,NPS / Kristi Rugg,Heavy snow-laden trees,Hiking tracks carved through three feet of sno...,During the colder months snows transform our l...,https://www.nps.gov/common/uploads/structured_...,NPS / Kristi Rugg,Sunset atop Cadillac Mountain,"A brilliant sunset filled with hues of blue, r...",As the tallest point on the eastern seaboard C...,https://www.nps.gov/common/uploads/structured_...,NPS / Kristi Rugg,Climbing The Precipice,Two hikers ascend a sheer cliff trail by way o...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,36240051-018E-4915-B6EA-3F1A7F24FBE4,https://www.nps.gov/arch/index.htm,Arches National Park,arch,Visit Arches to discover a landscape of contra...,38.72261844,-109.5863666,UT,4357192299,archinfo@nps.gov,30.00,25.00,15.00,55.00,Arches National Park is located in southeast U...,http://www.nps.gov/arch/planyourvisit/directio...,84532,Moab,UT,"5 miles north of Moab, Utah, on US 191",NaN,NaN,NPS/Neal Herbert,The Organ with Potholes,a stone monolith reflected in standing water,The Organ rock formation is reflected in one o...,https://www.nps.gov/common/uploads/structured_...,NPS Photo,Double O Arch,"a broad, red arch with rock pinnacles in the b...",Double O Arch is one of many large arches in t...,https://www.nps.gov/common/uploads/structured_...,NPS Photo,Delicate Arch,a stone arch,Delicate Arch is perhaps the most famous natur...,https://www.nps.gov/common/uploads/structured_...,NPS/Wonderly,Milky Way over the Garden of Eden,the Milky Way arcs above silhouetted stone pin...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,B170CCF7-7AB9-48FF-950E-31815FD4DBB2,https://www.nps.gov/badl/index.htm,Badlands National Park,badl,The rugged beauty of the Badlands draws visito...,43.68584846,-102.482942,SD,6054335361,badl_information@nps.gov,30.00,15.00,25.00,50.00,Badlands National Park is located 75 miles eas...,http://www.nps.gov/badl/planyourvisit/directio...,57750,Interior,SD,25216 Ben Reifel Road,N

### Create import template &&&

In [ ]:
# Write to the CSV import file
# activities_df.dropna()
activities_df.to_csv(r'/content/drive/Shareddrives/CIS 550 Project/Import Files/activities_import.csv', index=False)

activities_df = activities_df["activity"]
activities_df = activities_df.drop_duplicates()
# print(type(activities_df))
# sort the activites
activities_df.sort_values(ascending=True, inplace=True)

# activities_df.dropna()
activities_df.to_csv(r'/content/drive/Shareddrives/CIS 550 Project/Import Files/unique_activities_import.csv', index=False)

## Extracting image data out of the spreadsheet style dataframe into images dataframe

In [ ]:
image_cols = ["parkcode"]
image_cols.extend([col for col in parks_info if 'images' in col]) 
# print(image_cols)
images = parks_info[image_cols]
images.head()
images_df = pd.DataFrame()
for i in range(30):
  image_title = "images_" + str(i) + "_title"
  image_url = "images_" + str(i) + "_url"
  image_altText = "images_" + str(i) + "_altText"
  image_caption = "images_" + str(i) + "_caption"
  temp = images[['parkcode', image_title, image_url, image_altText, image_caption]]
  temp = temp.rename(columns={image_title: "title", image_url: "url", image_altText: "altText", image_caption:'caption'})
  # print(temp)
  images_df = images_df.append(temp)

images_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
images_df.sort_values(by=['parkcode', 'title'], inplace=True)
images_df.head(25)  

,parkcode,title,url,altText,caption
0,acad,Acadia's rocky coastline,https://www.nps.gov/common/uploads/structured_...,Large puffy clouds dot a brilliant blue sky as...,Millions of people come to Acadia for our dist...
0,acad,Climbing The Precipice,https://www.nps.gov/common/uploads/structured_...,Two hikers ascend a sheer cliff trail by way o...,Whether it's a stroll along Ocean Path or a di...
0,acad,Heavy snow-laden trees,https://www.nps.gov/common/uploads/structured_...,Hiking tracks carved through three feet of sno...,During the colder months snows transform our l...
0,acad,Sunset atop Cadillac Mountain,https://www.nps.gov/common/uploads/structured_...,"A brilliant sunset filled with hues of blue, r...",As the tallest point on the eastern seaboard C...
0,arch,Delicate Arch,https://www.nps.gov/common/uploads/structured_...,a stone arch,Delicate Arch is perhaps the most famous natur...
0,arch,Double O Arch,https://www.nps.gov/common/uploads/structured_...,"a broad, red arch with rock pinnacles in the b...",Double O Arch is one of many large arches in t...
0,arch,Milky Way over the Garden of Eden,https://www.nps.gov/common/uploads/structured_...,the Milky Way arcs above silhouetted stone pin...,Arches offers some excellent night sky viewing.
0,arch,Park Avenue Trail,https://www.nps.gov/common/uploads/structured_...,two hikers descend a broad wash with tall rock...,The Park Avenue trail is one of many hiking tr...
0,arch,The Organ with Potholes,https://www.nps.gov/common/uploads/structured_...,a stone monolith reflected in standing water,The Organ rock formation is reflected in one o...
0,badl,Badlands Rugged Peaks,https://www.nps.gov/common/uploads/structured_...,Badlands formations are very rugged and often ...,Badlands formations are very rugged and often ...


In [ ]:
images_df.head(25)

,parkcode,title,url,altText,caption
0,acad,Acadia's rocky coastline,https://www.nps.gov/common/uploads/structured_...,Large puffy clouds dot a brilliant blue sky as...,Millions of people come to Acadia for our dist...
0,acad,Climbing The Precipice,https://www.nps.gov/common/uploads/structured_...,Two hikers ascend a sheer cliff trail by way o...,Whether it's a stroll along Ocean Path or a di...
0,acad,Heavy snow-laden trees,https://www.nps.gov/common/uploads/structured_...,Hiking tracks carved through three feet of sno...,During the colder months snows transform our l...
0,acad,Sunset atop Cadillac Mountain,https://www.nps.gov/common/uploads/structured_...,"A brilliant sunset filled with hues of blue, r...",As the tallest point on the eastern seaboard C...
0,arch,Delicate Arch,https://www.nps.gov/common/uploads/structured_...,a stone arch,Delicate Arch is perhaps the most famous natur...
0,arch,Double O Arch,https://www.nps.gov/common/uploads/structured_...,"a broad, red arch with rock pinnacles in the b...",Double O Arch is one of many large arches in t...
0,arch,Milky Way over the Garden of Eden,https://www.nps.gov/common/uploads/structured_...,the Milky Way arcs above silhouetted stone pin...,Arches offers some excellent night sky viewing.
0,arch,Park Avenue Trail,https://www.nps.gov/common/uploads/structured_...,two hikers descend a broad wash with tall rock...,The Park Avenue trail is one of many hiking tr...
0,arch,The Organ with Potholes,https://www.nps.gov/common/uploads/structured_...,a stone monolith reflected in standing water,The Organ rock formation is reflected in one o...
0,badl,Badlands Rugged Peaks,https://www.nps.gov/common/uploads/structured_...,Badlands formations are very rugged and often ...,Badlands formations are very rugged and often ...


In [ ]:
parks_info = parks_info[parks_info.columns.drop(list(parks_info.filter(regex='images')))]
parks_info.drop(columns=["name","designation"],inplace=True)
parks_info['description'] = parks_info['description'].map(lambda x: re.sub(r'[^\w\s]', '', x))
parks_info['weather'] = parks_info['description'].map(lambda x: re.sub(r'[^\w\s]', '', x))
parks_info.sort_values(by=['parkcode'], inplace=True)
parks_info.head()


,id,website,parkname,parkcode,description,latitude,longitude,states,phone_number,email,entrance_fees_vehicle,entrance_fees_motorcycle,entrance_fees_individuals,entrance_pass_fees,direction,directionsUrl,zipcode,city,state,address_line_1,address_line_3,address_line_2,weather
0,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,https://www.nps.gov/acad/index.htm,Acadia National Park,acad,Acadia National Park protects the natural beau...,44.409286,-68.247501,ME,2072883338,acadia_information@nps.gov,30.00,25.00,15.00,55.00,"From Boston take I-95 north to Augusta, Maine,...",http://www.nps.gov/acad/planyourvisit/directio...,04609,Bar Harbor,ME,25 Visitor Center Road,NaN,Hulls Cove Visitor Center,Acadia National Park protects the natural beau...
0,36240051-018E-4915-B6EA-3F1A7F24FBE4,https://www.nps.gov/arch/index.htm,Arches National Park,arch,Visit Arches to discover a landscape of contra...,38.72261844,-109.5863666,UT,4357192299,archinfo@nps.gov,30.00,25.00,15.00,55.00,Arches National Park is located in southeast U...,http://www.nps.gov/arch/planyourvisit/directio...,84532,Moab,UT,"5 miles north of Moab, Utah, on US 191",NaN,NaN,Visit Arches to discover a landscape of contra...
0,B170CCF7-7AB9-48FF-950E-31815FD4DBB2,https://www.nps.gov/badl/index.htm,Badlands National Park,badl,The rugged beauty of the Badlands draws visito...,43.68584846,-102.482942,SD,6054335361,badl_information@nps.gov,30.00,15.00,25.00,50.00,Badlands National Park is located 75 miles eas...,http://www.nps.gov/badl/planyourvisit/directio...,57750,Interior,SD,25216 Ben Reifel Road,NaN,NaN,The rugged beauty of the Badlands draws visito...
0,C9056F71-7162-4208-8AE9-2D0AEFA594FD,https://www.nps.gov/bibe/index.htm,Big Bend National Park,bibe,There is a place in Far West Texas where night...,29.29817767,-103.2297897,TX,432-477-2251,bibe_info@nps.gov,30.00,25.00,12.00,55.00,Several highways lead to Big Bend National Par...,http://www.nps.gov/bibe/planyourvisit/directio...,79834,Big Bend National Park,TX,1 Panther Junction,NaN,NaN,There is a place in Far West Texas where night...
0,FBF9F793-5114-4B61-A5BA-6F9ADDFDF459,https://www.nps.gov/bisc/index.htm,Biscayne National Park,bisc,Within sight of downtown Miami yet worlds away...,25.490587,-80.21023851,FL,3052301144,bisc_information@nps.gov,0.00,NaN,NaN,0.00,The Dante Fascell Visitor Center may be reache...,http://www.nps.gov/bisc/planyourvisit/directio...,33033,Homestead,FL,9700 SW 328th Street,NaN,Sir Lancelot Jones Way,Within sight of downtown Miami yet worlds away...


### Create import template

In [ ]:
# parks_info[['address_line_2', 'address_line_3']].fillna(value='N/A', inplace=True)
parks_info['address_line_2'] = parks_info['address_line_2'].fillna('N/A')
parks_info['address_line_3'] = parks_info['address_line_3'].fillna('N/A')

# Write to the CSV import file
parks_info.to_csv(r'/content/drive/Shareddrives/CIS 550 Project/Import Files/parks_import.csv', index=False)
images_df.to_csv(r'/content/drive/Shareddrives/CIS 550 Project/Import Files/images_import.csv', index=False)

## Extracting state(s) where the park is located out of the main dataframe into states dataframe

In [ ]:
park_states = parks_info[["parkcode", "states"]]
parks_info = parks_info.columns.drop("states")
# source: https://stackoverflow.com/questions/12680754/split-explode-pandas-dataframe-string-entry-to-separate-rows

park_states = pd.concat([pd.Series(row['parkcode'], row['states'].split(','))              
                    for _, row in park_states.iterrows()]).reset_index()
park_states.rename(columns={"index": "state", 0: "parkcode"}, inplace=True)
# print(park_states.columns)
# print(type(park_states))
park_states =  park_states[["parkcode","state"]]
park_states.sort_values(by=["parkcode", "state"], inplace=True)
park_states.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
park_states.head(25)

,parkcode,state
0,acad,ME
1,arch,UT
2,badl,SD
3,bibe,TX
4,bisc,FL
5,blca,CO
6,brca,UT
7,cany,UT
8,care,UT
9,cave,NM


## Create import template

In [ ]:
# Write to the CSV import file
park_states.to_csv(r'/content/drive/Shareddrives/CIS 550 Project/Import Files/states_import.csv',index=False)